In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"mahighosh149","key":"de64ee62ca749c63f5f1fc960358f419"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d tongpython/cat-and-dog

Dataset URL: https://www.kaggle.com/datasets/tongpython/cat-and-dog
License(s): CC0-1.0
cat-and-dog.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip cat-and-dog.zip -d ./dataset

Archive:  cat-and-dog.zip
replace ./dataset/test_set/test_set/cats/_DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import SelectKBest, f_classif


In [ ]:
train_dir = '/content/dataset/training_set/training_set'
test_dir = '/content/dataset/test_set/test_set'

In [ ]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for subfolder in ['cats', 'dogs']:
        path = os.path.join(folder, subfolder)
        for filename in os.listdir(path):
            img_path = os.path.join(path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (64, 64))
                images.append(img)
                labels.append(label[subfolder])
    return np.array(images), np.array(labels)

label_map = {'cats': 0, 'dogs': 1}
X_train, y_train = load_images_from_folder(train_dir, label_map)
X_test, y_test = load_images_from_folder(test_dir, label_map)


In [ ]:
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)


In [ ]:
from sklearn.model_selection import train_test_split

X_train_small, _, y_train_small, _ = train_test_split(X_train_scaled, y_train, test_size=0.9, random_state=42)
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train_small, y_train_small)


SVC(kernel='linear', random_state=42)

In [ ]:
y_pred = svm_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(classification_report(y_test, y_pred, target_names=['Cat', 'Dog']))


Accuracy: 54.47%
              precision    recall  f1-score   support

         Cat       0.54      0.60      0.57      1011
         Dog       0.55      0.49      0.52      1012

    accuracy                           0.54      2023
   macro avg       0.55      0.54      0.54      2023
weighted avg       0.55      0.54      0.54      2023



In [ ]:
def predict_image_svm(filepath, model, scaler):
    img = cv2.imread(filepath)
    img = cv2.resize(img, (64, 64))
    img_flat = img.reshape(1, -1)
    img_scaled = scaler.transform(img_flat)

    prediction = model.predict(img_scaled)
    if prediction[0] == 1:
        print("The image is of a Dog")
    else:
        print("The image is of a Cat")

predict_image_svm('/content/Dog.jpeg', svm_model, scaler)


The image is of a Dog
